# 提供信息

In [1]:
bar_csv = 'barcode.csv'

amino_csv = '氨基酸表可用.csv'

com_fq = 'D-1.fq'

amino_left1 = 'aggtggag' 
amino_right1 = 'acagccca'
amino_len1 = 147

# amino_left2 = 'gatgcctttttc' 
# amino_right2 = 'cccaccagtcag'
# amino_len2 = 15

zamino_len = 49



N20_left = 'cgagtcacca'
N20_right = 'tgaggtggag'
N20line = 'aagaagaagcggaaagtctc'
N20_len = 20
Before = 'A'
After = 'G'
pos_list = [1,2,4,5,7,8,13,14,15]



linelen = 10
Dreads = 1

# lr_txt = 'indel.txt'
# mediant = 20 # 2.中间长度


# 固定代码

### 导入库

In [1]:
import pandas as pd
import re
import os
import operator
from Bio.Seq import Seq
from fuzzysearch import find_near_matches
from difflib import SequenceMatcher
from Bio import pairwise2
import csv

def complement(sequence):
    sequence = sequence.upper()
    sequence = sequence.replace('A', 't')
    sequence = sequence.replace('T', 'a')
    sequence = sequence.replace('C', 'g')
    sequence = sequence.replace('G', 'c')
    return sequence.upper()

def reverse(sequence):
    sequence = sequence.upper()
    return sequence[::-1]

def complement_reverse(sequence):
    return complement(reverse(sequence))

def mkdir(path):
    path=path.strip()
    path=path.rstrip("\\")
    isExists=os.path.exists(path)
    if not isExists:
        os.makedirs(path) 
#         print(path+' 创建成功')
#         return True
#     else:
#         print(path+' 目录已存在')
#         return False

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

import time

start_time = time.time()

In [2]:
N20line='ACGTAA'
n20='ACTA'
alignments = pairwise2.align.globalms(N20line, n20, 5, -4, -5, -.1,one_alignment_only = True)
alignments

[Alignment(seqA='ACGTAA', seqB='AC-T-A', score=10.0, start=0, end=6)]

### 根据barcode分组并且删除小于条数的序列

In [3]:
df = pd.read_csv(bar_csv, index_col=None, header=0,sep = ',',encoding="utf-8")

add = 0
n_list = []#存初始名字
for i in range(df.shape[0]):#range(df.shape[0])
#     p=0
    rf = open("./{}".format(com_fq))
    name = df['name'][i]
    n_list.append(name)
    name_2 = name[0:name.rfind('-')]
    mkdir('./indel数据/{}'.format(name_2))
    barcode = df['barcode'][i].upper()   #大写
    re_barcode = complement_reverse(barcode)
    print (name,barcode,re_barcode)
###     for j in range(groups):
###         j+=1
###         if '{}_'.format(j) in name:
    A1 = open("./indel数据/{}/{}.txt".format(name_2,name),'w')
    for line in rf:
#        p+=1
        line = line.strip()
        if ( barcode in line[:20]):   #line[:20]没固定barcode
    #         print(line)
            A1.write(line)
            A1.write('\n')
        elif ( re_barcode in line[-20:]):
    #         print(complement_reverse(line))
            A1.write(complement_reverse(line))
            A1.write('\n')
#         if p==100000:
#             break
    A1.close()
    A2 = open("./indel数据/{}/{}.txt".format(name_2,name))
    count_dict = {}
    for line2 in A2:
        line2 = line2.rstrip()#消除/n
        count = count_dict.setdefault(line2, 0)
        count += 1
        count_dict[line2] = count
    sorted_count_dict = sorted(count_dict.items(), key=operator.itemgetter(1), reverse=True)
    A3 = open("./indel数据/{}/{}.txt".format(name_2,name),'w')
    A3.write('Grouping sequence'+'\t'+'\t'+'\t'+'Count')
    A3.write('\n')
    aaa=0
    for item in sorted_count_dict:
        aaa+=1
        if item[1] >= linelen:
            A3.write(item[0]+'\t'+str(item[1]))
            add += item[1]
            A3.write('\n')
        if aaa%100000 == 0:
            print(aaa)
    print(i+1)
    A2.close()
    A3.close()
#     A4 = open("./indel数据/{}/{}.txt".format(name[:-2],name))
#     for ii in A4:
#         print(ii)
    rf.close()

A1 GCGAGTTCGA TCGAACTCGC
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
1


In [4]:
add

8129374

## 先序列

In [5]:
Lline = open('./indel数据/A/A1.txt') 

add1=0
n20_list = []
line_list = []
reads_list = []
a = 0
x = 0
for l in Lline:
    if x>0:
        l = l.strip()
        #print (line)
        segments = l.split('	')
        line = segments[0]
        reads = segments[1]
#         if int(reads)<Amilen:
#             continue
        
        #print (segments)
        #print(b)
    #         print('sequence: ', line) 
        N20_left1 = N20_left.upper()
        N20_right1 = N20_right.upper()
       # print(line)

        N20_left_index = find_near_matches(N20_left1, line, max_l_dist=0)
        if N20_left_index:
            N20_right_index = find_near_matches(N20_right1, line, max_l_dist=0)
            if N20_right_index:          
                N20_left_end = N20_left_index[0].end    
                N20_right_start = N20_right_index[0].start

                n20 = line[N20_left_end:N20_right_start]
                if len(n20) != N20_len:
                    continue


                


#         amino_left2 = amino_left2.upper()
#         amino_right2 = amino_right2.upper()
#        # print(line)

#         N20_left_index = find_near_matches(amino_left2, line, max_l_dist=0)
#         if N20_left_index:
#             N20_right_index = find_near_matches(amino_right2, line, max_l_dist=0)
#             if N20_right_index:          
#                 N20_left_end = N20_left_index[0].end    
#                 N20_right_start = N20_right_index[0].start

#                 n202 = line[N20_left_end:N20_right_start]
#                 if len(n202) != amino_len2:
#                     continue
#                 substring1 = n202[0:3]  
#                 substring2 = n202[6:9]  
#                 n202 = substring1 + substring2
                n20_list.append(n20)
                line_list.append(line)
                reads_list.append(reads)
                add1 += int(reads)

    x+=1
df = pd.DataFrame({
    'n20' : n20_list,
    'line' : line_list,
    'reads' : reads_list
})

df

,n20,line,reads
0,AAGAAGAAGCGGAAAGTCTC,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,1894247
1,AAGAAGAAGCGGAAAGTCTC,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,278854
2,AAGGGGGGGCGGAAAGTCTC,ACGGAAGCGAGTTCGAGTCACCAAAGGGGGGGCGGAAAGTCTCTGA...,109713
3,AAGGGGGAGCGGAAAGTCTC,ACGGAAGCGAGTTCGAGTCACCAAAGGGGGAGCGGAAAGTCTCTGA...,55799
4,AAGAAGAAGCGGAAAGTCTC,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,30817
...,...,...,...
56255,AAGAAGAAGCGGAAAGTCTC,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,10
56256,AAGGGGGGGCGGAAAGTCTC,ACGGAAGCGAGTTCGAGTCACCAAAGGGGGGGCGGAAAGTCTCTGA...,10
56257,AAGAAGAAGCGGAAAGTCTC,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,10
56258,AAGAAGAAGCGGAAAGTCTC,GGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGAGG...,10


In [6]:
# 使用groupby和filter方法保留数量加和大于100的组
df['reads'] = df['reads'].astype(int)
df_filtered = df.groupby('n20').filter(lambda x: x['reads'].sum() > Dreads)
df_filtered = df_filtered.drop('n20', axis=1)

# 将数据写入到TXT文件中，使用制表符分隔不同的列
df_filtered.to_csv('./indel数据/A/A2.txt', sep='\t', index=False)

df_filtered

,line,reads
0,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,1894247
1,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,278854
2,ACGGAAGCGAGTTCGAGTCACCAAAGGGGGGGCGGAAAGTCTCTGA...,109713
3,ACGGAAGCGAGTTCGAGTCACCAAAGGGGGAGCGGAAAGTCTCTGA...,55799
4,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,30817
...,...,...
56255,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,10
56256,ACGGAAGCGAGTTCGAGTCACCAAAGGGGGGGCGGAAAGTCTCTGA...,10
56257,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,10
56258,GGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGAGG...,10


### 挑选出含氨基酸的序列

In [7]:
Lline = open('./indel数据/A/A2.txt') 

add1=0
n20_list = []
line_list = []
reads_list = []
a = 0
x = 0
for l in Lline:
    if x>0:
        l = l.strip()
        #print (line)
        segments = l.split('	')
        line = segments[0]
        reads = segments[1]
#         if int(reads)<Amilen:
#             continue
        
        #print (segments)
        #print(b)
    #         print('sequence: ', line) 
        amino_left1 = amino_left1.upper()
        amino_right1 = amino_right1.upper()
       # print(line)

        N20_left_index = find_near_matches(amino_left1, line, max_l_dist=0)
        if N20_left_index:
            N20_right_index = find_near_matches(amino_right1, line, max_l_dist=0)
            if N20_right_index:          
                N20_left_end = N20_left_index[0].end    
                N20_right_start = N20_right_index[0].start

                n20 = line[N20_left_end:N20_right_start]
                if len(n20) != amino_len1:
                    continue


                


#         amino_left2 = amino_left2.upper()
#         amino_right2 = amino_right2.upper()
#        # print(line)

#         N20_left_index = find_near_matches(amino_left2, line, max_l_dist=0)
#         if N20_left_index:
#             N20_right_index = find_near_matches(amino_right2, line, max_l_dist=0)
#             if N20_right_index:          
#                 N20_left_end = N20_left_index[0].end    
#                 N20_right_start = N20_right_index[0].start

#                 n202 = line[N20_left_end:N20_right_start]
#                 if len(n202) != amino_len2:
#                     continue
#                 substring1 = n202[0:3]  
#                 substring2 = n202[6:9]  
#                 n202 = substring1 + substring2
                n20_list.append(n20)
                line_list.append(line)
                reads_list.append(reads)
                add1 += int(reads)

    x+=1
df = pd.DataFrame({
    'amino' : n20_list,
    'line' : line_list,
    'reads' : reads_list
})

df

,amino,line,reads
0,TTTTCCCACGAGTACTGGATGAGACATGCCCTGACCCTGGCCAAGA...,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,1894247
1,TTCTCCCACGAGTACTGGATGAGACATGCCCTGACCCCGGCCAAGA...,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,278854
2,TTTTCCCACGAGTACTGGATGAGACATGCCCTGACCCTGGCCAAGA...,ACGGAAGCGAGTTCGAGTCACCAAAGGGGGGGCGGAAAGTCTCTGA...,109713
3,TTTTCCCACGAGTACTGGATGAGACATGCCCTGACCCTGGCCAAGA...,ACGGAAGCGAGTTCGAGTCACCAAAGGGGGAGCGGAAAGTCTCTGA...,55799
4,TTTTCCCACGAGTACTGGATGAGACATGCCCTGACCCTGGCCAAGA...,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,30817
...,...,...,...
46830,TCTTCCCACGAGTACTGGATGAGACATGCCCTGACCCTGGCCAAGA...,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,10
46831,TTTTCCCACGAGTACTGGGTGAGACATGCCCTGACCCTGGCCAAGA...,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,10
46832,TTTTCCCACGAGTACTGGATGAGACATGCCCTGACCCTGGCCAAGA...,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,10
46833,TTTTCCCACGAGTACTGGATGAGACATGCCCTGACCCTGGCCAAGA...,GGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGAGG...,10


In [8]:
add1

6674375

In [9]:
add1

6674375

### 根基氨基酸表确认类型

In [10]:
def translate_dna_to_amino_acids(dna_sequence):
    genetic_code = {
        'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L',
        'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L',
        'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M',
        'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V',
        'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
        'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
        'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
        'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
        'TAT': 'Y', 'TAC': 'Y', 'TAA': 'Z', 'TAG': 'Z',
        'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
        'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
        'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
        'TGT': 'C', 'TGC': 'C', 'TGA': 'Z', 'TGG': 'W',
        'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
        'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
        'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'
    }
    
    amino_acids = []
    codon = ''
    
    for base in dna_sequence:
        codon += base
        if len(codon) == 3:
            amino_acid = genetic_code.get(codon, 'X')  # 'X' represents unknown amino acid
            amino_acids.append(amino_acid)
            codon = ''
    
    return ''.join(amino_acids)


AmiMer = open('./AmiMer.txt','w')
for dna_sequence in df['amino']:
    amino_acid_sequence = translate_dna_to_amino_acids(dna_sequence)
    AmiMer.write(amino_acid_sequence)
    AmiMer.write('\n')
AmiMer.close()

AmiMer_list = []
adt = open('./AmiMer.txt')
for adl in adt:
    adl = adl.strip('\n')
    #print (adl)
    AmiMer_list.append(adl)
adt.close()

#df['AmiMer'] = AmiMer_list
df.insert(0, 'AmiMer', AmiMer_list)
df = df.drop('amino', axis=1)
# AmiMerdf = pd.DataFrame({
#     'AmiMer' : AmiMer_list
# })
df



,AmiMer,line,reads
0,FSHEYWMRHALTLAKRARDEREVPVGAVLVLNNRVIGEGWNRAIGLHDP,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,1894247
1,FSHEYWMRHALTPAKRARDEREVPVGAVLVLNNSVIGEGWNRAIGLHDP,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,278854
2,FSHEYWMRHALTLAKRARDEREVPVGAVLVLNNRVIGEGWNRAIGLHDP,ACGGAAGCGAGTTCGAGTCACCAAAGGGGGGGCGGAAAGTCTCTGA...,109713
3,FSHEYWMRHALTLAKRARDEREVPVGAVLVLNNRVIGEGWNRAIGLHDP,ACGGAAGCGAGTTCGAGTCACCAAAGGGGGAGCGGAAAGTCTCTGA...,55799
4,FSHEYWMRHALTLAKRAREEREVPVGAVLVLNNRVIGEGWNRAIGLHDP,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,30817
...,...,...,...
46830,SSHEYWMRHALTLAKRARDEREVPVGVVLVLNNRVIGEGWNRAIGLHDP,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,10
46831,FSHEYWVRHALTLAKRARDERGVPVGAVLVLNNRVIGEGWNZAIGLHNP,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,10
46832,FSHEYWMRHALTLAKRARGEREVPEGAVLVLNNRVIGEGWNRAIGLHDP,ACGGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGA...,10
46833,FSHEYWMRHALTLAKRARDEREVPAGAVLVLNNRVIGEGWNRAIGLYDP,GGAAGCGAGTTCGAGTCACCAAAGAAGAAGCGGAAAGTCTCTGAGG...,10


### 去重保留所有氨基酸种类

In [11]:
# 根据'category'列进行分类
grouped = df.groupby('AmiMer')

data_dir = './amino'
if not os.path.exists(data_dir):  
    os.mkdir(data_dir)

n=0
for name, group in grouped:
    n+=1
    # 将DataFrame保存为txt文件
    group.to_csv('./amino/{}.txt'.format(name), header=False, index=False, sep='\t')
    if n%50000 == 0:
        print(n)

### 定位N20，计算indel，将N20拆分

In [12]:
data_dir = './reads'
if not os.path.exists(data_dir):  
    os.mkdir(data_dir)
    
data_dir = './indel'
if not os.path.exists(data_dir):  
    os.mkdir(data_dir)

path = "./amino"
dirs = os.listdir( path )
name_list = []
for i in range(0,len(dirs)): #9.循环，不用顾虑个数
    A = dirs[i]
    name_list.append(A)

result = open('./indel/result.txt','w')

for n in range(0,len(dirs)):
    df1 = open('./amino/%s'%(name_list[n]))
    eff = open('./reads/result_%s'%(name_list[n]),'w')
    #TCCAGCCCAGATCTATAGGG
    eff.write('pos1'+'\t'+'reads'+'\t'+'pos2'+'\t'+'reads'+'\t'+'pos3'+'\t'+'reads'+'\t'+'pos4'+'\t'+'reads'+'\t'+'pos5'+'\t'+'reads'
              +'\t'+'pos6'+'\t'+'reads'+'\t'+'pos7'+'\t'+'reads'+'\t'+'pos8'+'\t'+'reads'+'\t'+'pos9'+'\t'+'reads'+'\t'+'pos10'+'\t'
              +'reads'+'\t'+'pos11'+'\t'+'reads'+'\t'+'pos12'+'\t'+'reads'+'\t'+'pos13'+'\t'+'reads'+'\t'+'pos14'+'\t'+'reads'
             +'\t'+'pos15'+'\t'+'reads'+'\t'+'pos16'+'\t'+'reads'+'\t'+'pos17'+'\t'+'reads'+'\t'+'pos18'+'\t'+'reads'
             +'\t'+'pos19'+'\t'+'reads'+'\t'+'pos20'+'\t'+'reads')
    x = 0
    eff.write('\n')
    
    delete_count,insert_count,equal_count = 0,0,0
    for l in df1:
        
        l = l.strip()
        segments = l.split('	')
        line = segments[1]
        #print (line)
        reads = segments[2]
        #print (reads)
        N20_left = N20_left.upper()
        #print(N20_left)
        N20_right = N20_right.upper()
        N20line = N20line.upper()

        
        N20_left_index = find_near_matches(N20_left, line, max_l_dist=0)
        if N20_left_index:
            #print('a')
            N20_right_index = find_near_matches(N20_right, line, max_l_dist=0)
            if N20_right_index:          
                N20_left_end = N20_left_index[0].end    
                N20_right_start = N20_right_index[0].start

                n20 = line[N20_left_end:N20_right_start]#这个n20算indel
                n20 = n20.replace('-', '')
                n20_len = len(n20)#去除前后-，是否为20
                #print(n20)
                #print(len(n20))
                if n20_len<N20_len:
                    delete_count += int(reads)
                elif n20_len == N20_len:
                    equal_count += int(reads)
                else:
                    if n20_len>N20_len:
                        insert_count += int(reads)
                        
                if len(n20)==0:
                    continue
                
                alignments = pairwise2.align.globalms(N20line, n20, 5, -4, -5, -.1,one_alignment_only = True)
                
                #alignments[-1][0],alignments[-1][1]
                #print(alignments)
                #print(n20)
                n20 = alignments[-1][1]#这个n20算效率
                #print(n20)
                
#                 if len(n20) != 20:
#                     continue
                if n20 == '':
                    continue
               
                for p in range(0,20):
                    if n20[p]=='A':
                        eff.write('A'+'\t'+reads+'\t')
                    elif n20[p]=='C':
                        eff.write('C'+'\t'+reads+'\t')
                    elif n20[p]=='G':
                        eff.write('G'+'\t'+reads+'\t')
                    elif n20[p]=='T':
                        eff.write('T'+'\t'+reads+'\t')
                    else:
                        eff.write('N'+'\t'+reads+'\t')
            else:
                continue
        else:
            continue
        eff.write('\n')




    total = delete_count+insert_count+equal_count
    if total==0:# 4.加if，防止程序终止
        #print('这一组中存在异常')
        result.write('name,delete_count,insert_count,total,delete_ratio,insert_ratio,sum_ratio')
        result.write('\n')
        result.write(name_list[n]+','+'存在异常'+','+'存在异常'+','+'存在异常'+','+'存在异常'+','+'存在异常'+','+'存在异常')
        result.write('\n')
        result.write('\n')
    else:
        result.write('name,delete_count,insert_count,total,delete_ratio,insert_ratio,sum_ratio')
        result.write('\n')
        result.write(name_list[n]+','+str(delete_count)+','+str(insert_count)+','+str(total)+','+str(delete_count/total)+','+str(insert_count/total)+','+str(delete_count/total+insert_count/total))
        result.write('\n')
        result.write('\n')
        
    if n%50000 == 0:
        print(n)
        
    df1.close()
    eff.close()
result.close()
                    
resulttxt = './indel/result.txt'  #以csv结果呈现
resultcsv = './indel/indel_result.csv'

path = './{}'.format(resulttxt)
f = open('./{}'.format(resultcsv),'w',encoding='utf-8',newline='')

csv_writer = csv.writer(f)
with open(path,'r') as f1:
    lines = f1.readlines()
    i = 0
    for line in lines:
        if i==0 or i%3==1:
            data = line.split(',')
            #print(data)
            data1 = data[6][:-1]
            csv_writer.writerow([data[0],data[1],data[2],data[3],data[4],data[5],data1])
        i+=1
f.close()                   

0


### 统计每位的碱基及数目

In [13]:
data_dir = './eff'
if not os.path.exists(data_dir):  
    os.mkdir(data_dir)

path = "./reads"
dirs = os.listdir( path )
name_list = []
for i in range(0,len(dirs)): #9.循环，不用顾虑个数
    A = dirs[i]
    name_list.append(A)


for n in range(0,len(dirs)):
    eff = open('./eff/eff_%s'%(name_list[n]),'w')
    
    
    eff.write('A'+'\t'+'C'+'\t'+'G'+'\t'+'T'+'\t'+'N'+'\t'+'total')
    for w in range(0,40,2):
        #print(w)
        a,g,c,t,nn=0,0,0,0,0
        x = 0
        df1 = open('./reads/%s'%(name_list[n]))
        for l in df1:
            if x>0:
                l = l.strip()
                #print (line)
                segments = l.split('\t')
                #print(segments)
                #print(w)
                pos = segments[w]
                reads = segments[w+1]
                #print(w)
                #print(reads)
                if pos=='A':
                    a+=int(reads)
                if pos=='C':
                    c+=int(reads)
                if pos=='G':
                    g+=int(reads)
                if pos=='T':
                    t+=int(reads)
                if pos=='N':
                    nn+=int(reads)
            x+=1
        eff.write('\n')
        eff.write(str(a)+'\t'+str(c)+'\t'+str(g)+'\t'+str(t)+'\t'+str(nn)+'\t'+str(a+c+g+t+nn))
        df1.close()
    
    if n%50000 == 0:
        print(n)
        
        
    
    eff.close()
                    

                    
                    
#                     if len(n20)==0:
#                         continue
#                     n20_list.append(n20)
#                     line_list.append(line)
#                     reads_list.append(reads)
#         #             print(n20)
#         #             N20.write(n20)
#         #             N20.write('\n')
#                     a+=1
#                     #print(a)
#         #                 print('\n')
#         x+=1

0


### 计算碱基占比

In [14]:
data_dir = './exp'
if not os.path.exists(data_dir):  
    os.mkdir(data_dir)

path = "./eff"
dirs = os.listdir( path )
name_list = []
for i in range(0,len(dirs)): #9.循环，不用顾虑个数
    A = dirs[i]
    name_list.append(A)


for n in range(0,len(dirs)):
    exp = open('./exp/exp_%s'%(name_list[n]),'w')
    exp.write('Position'+'\t'+'\t'+'Aratio'+'\t'+'\t'+'Cratio'+'\t'+'\t'+'Gratio'+'\t'+'\t'+'Tratio'+'\t'+'\t'+'Total')
    exp.write('\n')
    
    x = 0
    df1 = open('./eff/%s'%(name_list[n]))
    for l in df1:
        if x>0:
            l = l.strip()
            #print (line)
            segments = l.split('\t')
            #print(segments)
            #print(w)
            A = segments[0]
            C = segments[1]
            G = segments[2]
            T = segments[3]
            N = segments[4]
            to = segments[5]
            sumacgt = int(A)+int(C)+int(G)+int(T)+int(N)
            #print(sumacgt)
            if sumacgt==0:
                sumacgt=1
            Aratio = int(A)/sumacgt
            Aratio = round(Aratio,12)
            Cratio = int(C)/sumacgt
            Cratio = round(Cratio,12)
            Gratio = int(G)/sumacgt
            Gratio = round(Gratio,12)
            Tratio = int(T)/sumacgt
            Tratio = round(Tratio,12)
            exp.write('Pos{}'.format(x)+'\t'+str(Aratio)+'\t'+str(Cratio)+'\t'+str(Gratio)+'\t'+str(Tratio)+'\t'+str(to))
            exp.write('\n')
        x+=1

    df1.close()
    
    if n%50000 == 0:
        print(n)
        
        
    
    exp.close()
                    

                    
                    


0


### 结果呈现为csv格式

In [15]:
data_dir = './eff_csv'
if not os.path.exists(data_dir):  
    os.mkdir(data_dir)

path = "./exp"
dirs = os.listdir( path )
name_list = []
for i in range(0,len(dirs)): #9.循环，不用顾虑个数
    A = dirs[i]
    name_list.append(A)
    



for n in range(0,len(dirs)):
    
    Alist = []
    Clist = []
    Glist = []
    Tlist = []
    Plist = []
    totlist = []
    df1 = open('./exp/%s'%(name_list[n]))
    #print(('./exp/%s'%(name_list[n])))
    x = 0
    for l in df1:
        if x>0:
            l = l.strip()
            #print (line)
            segments = l.split('\t')
           # print(segments)
            #print(w)
            A = segments[1]
            Alist.append(A)
            C = segments[2]
            Clist.append(C)
            G = segments[3]
            Glist.append(G)
            T = segments[4]
            Tlist.append(T)
            P = segments[0]
            Plist.append(P)
            tot = segments[5]
            totlist.append(tot)
        x+=1
    df2 = pd.DataFrame({
        'Position' : Plist,
        'A' : Alist,
        'C' : Clist,
        'G' : Glist,
        'T' : Tlist,
        'Total' : totlist
        })
    df2.to_csv('./eff_csv/{}.csv'.format(('./exp/%s'%(name_list[n]))[14:-4]),index=False)
        

### 挑选变化碱基的位置及效率

In [16]:
data_dir = './dre_csv'
if not os.path.exists(data_dir):  
    os.mkdir(data_dir)

path = "./eff_csv"
dirs = os.listdir( path )
name_list = []
for i in range(0,len(dirs)): #9.循环，不用顾虑个数
    A = dirs[i]
    name_list.append(A)
    
    

#pos2 = 8
N20line = N20line.upper()
Before = Before.upper()
After = After.upper()
j = 0
# pos_list = [2,5,8,10,11,14,15,16,20]
#pos_list.append(pos1)
#pos_list.append(pos2)
# for i in N20line:
#     j+=1
#     if i==Before:
#         pos_list.append(j)
        #print(pos_list[0])

dre = open('./dre_csv/dre.txt','w')
dre.write('amino'+'\t'+'\t')
for p in pos_list:
    dre.write('pos{}'.format(p)+'\t'+'\t')
dre.write('Total')
dre.write('\n')
for n in range(0,len(dirs)):
#     drelist = [] 
#     poslist = []
    #print('./eff_csv/%s'%(name_list[n]))
    df1 = pd.read_csv('./eff_csv/%s'%(name_list[n]))
    if df1['Total'][0]<100:
        continue
    q = 0
    dre.write(('./eff_csv/%s'%(name_list[n]))[17:-4]+'\t')
    for l in df1[After]:
        q+=1
        for p in pos_list:
            if q==p:
                dre.write(str(l)+'\t')
    dre.write(str(df1['Total'][0]))
    dre.write('\n')
dre.close()

csvFile = open("./dre_csv/dre.csv",'w',newline='',encoding='utf-8')
writer = csv.writer(csvFile)
csvRow = []

f = open("./dre_csv/dre.txt",'r')
for line in f:
    csvRow = line.split()
    writer.writerow(csvRow)

f.close()
csvFile.close()

In [17]:
end_time = time.time()
execution_time = end_time - start_time
print(f"代码运行时间为 {execution_time} 秒")

代码运行时间为 731.8780562877655 秒


In [18]:
execution_time/60/60

0.20329946007993488